In [2]:
# imports
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

In [3]:
df = pd.read_csv(r'C:\Users\Maple\Downloads\energy1.csv')
#df

In [4]:
# exploring all our variables
#df.columns

# determining dtypes
#df.dtypes

In [5]:
#df['COMMUNITY AREA NAME'].unique()

In [6]:
# Define function to plot based on building type

def plot(area_name):
    # Filter data for building type
    data = df[df['COMMUNITY AREA NAME'] == area_name]

    # month columns
    kwh_columns = ['2010-01-01 00:00:00', '2010-02-01 00:00:00',
       '2010-03-01 00:00:00', '2010-04-01 00:00:00', '2010-05-01 00:00:00',
       '2010-06-01 00:00:00', '2010-07-01 00:00:00', '2010-08-01 00:00:00',
       '2010-09-01 00:00:00', '2010-10-01 00:00:00', '2010-11-01 00:00:00']

    # Collecting the average kilowatt/hour for each month in 2010
    monthly_averages = data[kwh_columns].mean()

    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(monthly_averages)
    plt.title(f'Average kWh Usage by Month for {area_name}')
    plt.grid(True)
    plt.xticks(rotation=90)
    plt.show()

# Create a dropdown menu for selecting building type
dropdown = widgets.Dropdown(
    options=['Morgan Park', 'West Pullman', 'Portage Park', 'Edgewater',
       'Rogers Park', 'Lincoln Square', 'Lakeview', 'Uptown', 'Clearing',
       'Chicago Lawn', 'Forest Glen', 'Avondale', 'South Deering',
       'Edison Park', 'Mount Greenwood', 'West Elsdon', 'West Lawn',
       'West Town', 'Belmont Cragin', 'Humboldt Park', 'Archer Heights',
       'Near North Side', 'Norwood Park', 'South Lawndale', 'South Shore',
       'Near West Side', 'Woodlawn', 'Jefferson Park', 'Garfield Ridge',
       'Pullman', 'Auburn Gresham', 'Austin', 'Lincoln Park',
       'North Center', 'Dunning', 'West Englewood', 'Logan Square',
       'Lower West Side', 'Beverly', 'West Ridge', 'Douglas',
       'East Garfield Park', 'East Side', 'Englewood', 'Bridgeport',
       'Brighton Park', 'Greater Grand Crossing', 'Hegewisch',
       'Calumet Heights', 'Irving Park', 'Washington Heights', 'Ashburn',
       'Montclare', 'North Lawndale', 'Roseland', 'Hermosa', 'New City',
       'South Chicago', 'Chatham', 'McKinley Park', 'West Garfield Park',
       'North Park', 'Kenwood', 'Albany Park', 'Hyde Park', 'Avalon Park',
       'Grand Boulevard', 'Armour Square', 'Gage Park', 'Washington Park',
       'Loop'],
    description='Community Area Name:',
    style={'description_width': 'initial'}
)

# Link the dropdown to the plotting function
widgets.interactive(plot, area_name=dropdown)

interactive(children=(Dropdown(description='Community Area Name:', options=('Morgan Park', 'West Pullman', 'Po…

In [7]:
# prompt: take the code for the above plot and make it in altair with the interactivity

df_subset = df[['COMMUNITY AREA NAME', '2010-01-01 00:00:00', '2010-02-01 00:00:00',
       '2010-03-01 00:00:00', '2010-04-01 00:00:00', '2010-05-01 00:00:00',
       '2010-06-01 00:00:00', '2010-07-01 00:00:00', '2010-08-01 00:00:00',
       '2010-09-01 00:00:00', '2010-10-01 00:00:00', '2010-11-01 00:00:00']]

kwh_columns = ['2010-01-01 00:00:00', '2010-02-01 00:00:00',
       '2010-03-01 00:00:00', '2010-04-01 00:00:00', '2010-05-01 00:00:00',
       '2010-06-01 00:00:00', '2010-07-01 00:00:00', '2010-08-01 00:00:00',
       '2010-09-01 00:00:00', '2010-10-01 00:00:00', '2010-11-01 00:00:00']

# Reshape the data for Altair
df_melted = df_subset.melt(
    id_vars=['COMMUNITY AREA NAME'],
    value_vars=[col for col in df_subset.columns if '2010' in col],
    var_name='Month',
    value_name='kWh'
)

melt = df_melted.sample(n=5000)

In [8]:
# Create the Altair interactive chart
brush = alt.selection_interval(encodings=['x', 'y'])

alt.Chart(melt).mark_line().encode(
    x='Month:T',
    y='mean(kWh)',
    color='COMMUNITY AREA NAME:N',
    tooltip=['Month', 'mean(kWh)', 'COMMUNITY AREA NAME']
).properties(
    width=800,
    height=800,
    title='Average kWh Usage by Month'
).add_selection(
        brush
).interactive(bind_y=False)

C:\Users\Maple\AppData\Local\Temp\ipykernel_15704\492347913.py:13: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.Chart(...)

In [9]:
# Create a dropdown menu for selecting community area
community_dropdown = widgets.Dropdown(
    options=df['COMMUNITY AREA NAME'].unique().tolist(),  # Use unique community area names
    description='Community Area Name:',
    style={'description_width': 'initial'}
)


# Function to update the Altair chart based on the selected community area
def line(selected_area):
    # Filter the data based on the selected area
    filtered_df = df_melted[df_melted['COMMUNITY AREA NAME'] == selected_area]

    # Create the Altair chart
    chart = alt.Chart(filtered_df).mark_line(point=True).encode(
        x='Month:T',
        y='mean(kWh):Q',
        tooltip=['Month', 'mean(kWh)', 'COMMUNITY AREA NAME']
    ).properties(
        width=800,
        height=800,
        title=f'Average kWh Usage by Month for {selected_area}'
    )#.interactive(bind_y=False)
    #chart.display() #added this line so the chart is displayed in the output
    # Save to JSON 
    chart.save('interactComArea.json', format='json')
    return chart
    
# Updating chart dynamically 
def updateChart(selected_area):
    chart = line(selected_area)
    display(chart)

# Link the dropdown to the update_chart function
interact = widgets.interactive(updateChart, selected_area=community_dropdown)

# Display widget
display(interact)

interactive(children=(Dropdown(description='Community Area Name:', options=('Morgan Park', 'West Pullman', 'Po…